In [1]:
import os
import cv2
import numpy as np

In [2]:
#Label untuk kelas image
subjects = ["","Female","Male"]

In [3]:
#fungsi untuk mendeteksi wajah menggunakan openCV
def detect_face(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)#konversi gambar dari RGB to Greyscale
    #mengakses file .xml untuk metode LBP pada oprenCV
    face_cascade = cv2.CascadeClassifier('/Users/Alwi/Desktop/DIP/Fisher face/lbpcascade_frontalface.xml')
    #mendeteksi multiscale image hasil keluaran berupa wajah yang terdeteksi
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5);
    #if no faces are detected then return original img
    if (len(faces) == 0): #jika pada gambar tidak terdeteksi bentuk wajah maka return "none" 
        return None, None
    x, y, w, h = faces[0] #mengekstrak area dari wajah
    return gray[y:y+h, x:x+w] , faces[0] #yang diambil dari gambar hanyalah wajah 

In [4]:
def prepare_training_data(data_folder_path): #fungsi untuk mempersiapkan training data
    folder1 = os.listdir(data_folder_path) #mengakses folder yg berisi dataset training
    faces = [] #list face
    labels = [] #list label
     #mengakses gambar-gambar yg ada pada folder Data-Training
    for dirs in folder1:
        if dirs.startswith("f") or dirs.startswith("m"):
            if dirs == "female" :
                label = 1
            else:
                label = 2
            dir_path = data_folder_path + "/" + dirs
            folder2 = os.listdir (dir_path)
            for dir_name in folder2: #mengakses gambar-gambar yg ada pada folder female / folder male
                subject_dir_path = data_folder_path + "/" + dirs + "/" + dir_name
                subject_images_names = os.listdir(subject_dir_path)
                for image_name in subject_images_names: #mengakses setiap gambar
                    if image_name.startswith("."):
                        continue;
                    #image path = Data-Training\female\anpage\anpage.1.jpg
                    image_path = subject_dir_path + "/" + image_name

                    #membuka gambar sesuai image_path
                    image = cv2.imread(image_path)

                    #menampilkan gambar
                    cv2.imshow("Training on image...", image)
                    cv2.waitKey(100)

                    #memanggil fungsi deteksi wajah
                    face, rect = detect_face(image)
        
                    if face is not None: 
                        faceresize = cv2.resize(face, (100, 100)) #resize gambar training
                        #membuat array face dan label
                        faces.append(faceresize)
                        labels.append(label)
        else:
            continue;
    #menutup jendela yg kebuka pada saat fungsi persiapan training data berjalan
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    cv2.destroyAllWindows()
 
    return faces, labels

In [5]:
def draw_rectangle(img, rect): #membuat bound box bentuk segiempat pada muka yg terdeteksi
    (x, y, w, h) = rect
    cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)

def draw_text(img, text, x, y): #menampilkan text label pada muka yg terdeteksi sesuai posisi (x,y)
    cv2.putText(img, text, (x, y), cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 255, 0), 2)

In [6]:
#Proses Training Data. list labels dan faces punya ukuran yg sama, list face berisi wajah, list labels berupa nama kelas
print("Preparing data...")
faces, labels = prepare_training_data("Data-Training") #memanggil fungsi untuk mempersiapkan data
print("Data prepared")
 
#melihat jumlah face dan label pada training data
print("Total faces: ", len(faces))
print("Total labels: ", len(labels))
face_recognizer =  cv2.face.EigenFaceRecognizer_create() #melakukan face recognition menggunakan metode eigenface
face_recognizer.train(faces, np.array(labels))

Preparing data...


error: OpenCV(3.4.3) /Users/travis/build/skvark/opencv-python/opencv/modules/objdetect/src/cascadedetect.cpp:1698: error: (-215:Assertion failed) !empty() in function 'detectMultiScale'


In [ ]:
def testing(image):
    img = image.copy() #copy gambar agar gambar asli tidak berubah
    #mendeteksi wajah
    face, rect = detect_face(img)
    face = cv2.resize(face, (100, 100)) #resize gambar agar sesuai dengan size gambar pada training data 
    label= face_recognizer.predict(face) #prediksi wajah yang terdeteksi masuk kedalam subject female/male
    print(label)
    label_text = subjects[label[0]]#mengakses array subjects untuk mendapat label text dari wajah yg sudah terprediksi
    print ("Gender pada gambar adalah\n"+label_text)
    draw_rectangle(img, rect)#drawing bound pada wajah yang terdeteksi
    draw_text(img, label_text, rect[0], rect[1]-5) #draw label text pada wajah yang terdeteksi
    return img, label_text

In [ ]:
#Proses Testing/Prediksi Menggunakan Foto-Foto
print("Testing...")
#mengakses gambar yg akan diprediksi
test_img1 = cv2.imread("Data-Training/female/anpage/anpage.1.jpg") #gambar dari training data
test_img2 = cv2.imread("Data-Training/male/ajflem/ajflem1.jpg") #gambar dari training data
test_img3 = cv2.imread("test-data/prabowo_2.jpg") #gambar dari data test
test_img4 = cv2.imread("test-data/pakde_1.jpg") #gambar dari data test\
#test_img5 = cv2.imread("data-test/rupert.jpg") #gambar dari data test

print("Result:")
#memanggil fungsi testing
hasiltesting1, label_text1 = testing(test_img1)
hasiltesting2, label_text2 = testing(test_img2)
hasiltesting3, label_text3 = testing(test_img3)
hasiltesting4, label_text4 = testing(test_img4)
hasiltesting5, label_text5 = testing(test_img5)
#menampilkan hasil prediksi
cv2.imshow (("gambar ke 1 ("+label_text1+")"), hasiltesting1)
cv2.imshow (("gambar ke 2 ("+label_text2+")"), hasiltesting2)
cv2.imshow (("gambar ke 3 ("+label_text3+")"), hasiltesting3)
cv2.imshow (("gambar ke 4 ("+label_text4+")"), hasiltesting4)
cv2.imshow (("gambar ke 5 ("+label_text5+")"), hasiltesting5)

#mengakhiri program
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
#Proses Testing/Prediksi Menggunakan Web Camera
video_capture = cv2.VideoCapture(0) #menyalakan webcamera
#mengakses file .xml untuk metode LBP pada openCV
cascade = cv2.CascadeClassifier('/Users/Alwi/Desktop/DIP/Fisher face/lbpcascade_frontalface.xml')
while True:
    ret, frame = video_capture.read() #Mengcapture frame-frame video
    converted = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY) #konversi dari RGB to GRAYSCALE
    resize = cv2.resize(converted, (100, 100)) #resize frame
    faces = cascade.detectMultiScale(converted, 1.3, 5) #detect wajah
    for(x, y, w, h) in faces: 
        frame = cv2.rectangle(frame, (x,y),(x+w, y+h),(0,255,0),2) #Draw bounding box berbentuk persegi
        label = face_recognizer.predict(resize) #melakukan prediksi male/female
        label_text = subjects[label[0]] #memberi label female/male sesuai hasil prediksi
        frame = cv2.putText(frame,label_text,(x,y), cv2.FONT_HERSHEY_SIMPLEX, 1.5,(0,255,0),2) #menuliskan label pada frame
    cv2.imshow('video jadi', frame) #menampilkan video hasil deteksi
    
    if cv2.waitKey(1) & 0xFF == ord('q'): #Interrupt untuk mengakhiri program
        break
#mengakhiri program
video_capture.release()
cv2.destroyAllWindows()
del(video_capture)